In [1]:
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score

import joblib

from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin

from feature_engine.imputation import (
    CategoricalImputer,
    AddMissingIndicator,
    MeanMedianImputer)

from feature_engine.encoding import (
    RareLabelEncoder,
    OneHotEncoder
)

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(891, 12)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
def first_cabin(row):
    try: 
        return row.split()[0]
    except:
        return np.nan

df['Cabin'] = df['Cabin'].apply(first_cabin)

In [7]:
def title_(passenger):
    line = passenger 
    if re.search('Mrs',line):
        return 'Mrs'
    elif re.search('Mr',line):
        return 'Mr'
    elif re.search('Miss',line):
        return 'Miss'
    elif re.search('Master',line):
        return 'Master'
    else: 
        return 'Other'
    
df['Title'] = df['Name'].apply(title_)

In [8]:
df = df.drop(['Ticket','Name','PassengerId'],axis = 1)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,NaN,S,Mr
1,1,1,female,38.0,1,0,71.2833,C85,C,Mrs
2,1,3,female,26.0,0,0,7.9250,NaN,S,Miss
3,1,1,female,35.0,1,0,53.1000,C123,S,Mrs
4,0,3,male,35.0,0,0,8.0500,NaN,S,Mr


In [9]:
numeric_variables = ['Age','Parch', 'Fare']

cabin = ['Cabin']

categoric_variables = ['Sex', 'Embarked', 'Title','Cabin']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Survived',axis = 1),
    df['Survived'],
    test_size=0.2,
    random_state=5)

X_train.shape , X_test.shape

((712, 9), (179, 9))

In [11]:
#This is for Cabin.
class ExtractLetterTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self,variables):
        
        if not isinstance(variables,list):
            
            raise ValueError('variables should be a list')
            
        self.variables = variables
    
    def fit(self, X, y = None):
       
        return self
    
    def transform(self,X):
        
        X = X.copy()
        
        for feature in self.variables:
            
            X[feature] = X[feature].str[0]
        
        return X

In [12]:
titanic_pipeline = Pipeline([
    
    ('categorical_imputation', CategoricalImputer(
        imputation_method = 'missing', variables = categoric_variables 
    )),
    
    #adding missing indicator to numerical variables
    ('missing_imputation' , AddMissingIndicator(
        variables=numeric_variables
    )),
    
    #imputing numeric variables with the median
    ('median_imputation', MeanMedianImputer(
        imputation_method = 'median', variables = numeric_variables 
    )),
    
    #Extract letter from cabin
    
    ('extract_letter', ExtractLetterTransformer(
        variables = cabin
    )),
    
    #Categorical Encoding
    
    ('rare_label_categories', RareLabelEncoder(
        tol = 0.05, n_categories = 1, variables = categoric_variables
    )),
    
    ('categorical_encoder', OneHotEncoder(
        drop_last = True, variables = categoric_variables
    )),
    
    ('scaler', StandardScaler()),
    
    ('logit', LogisticRegression(C = 0.0005, random_state=0))
    
])

In [13]:
titanic_pipeline.fit(X_train,y_train)

Pipeline(steps=[('categorical_imputation',
                 CategoricalImputer(variables=['Sex', 'Embarked', 'Title',
                                               'Cabin'])),
                ('missing_imputation',
                 AddMissingIndicator(variables=['Age', 'Parch', 'Fare'])),
                ('median_imputation',
                 MeanMedianImputer(variables=['Age', 'Parch', 'Fare'])),
                ('extract_letter',
                 ExtractLetterTransformer(variables=['Cabin'])),
                ('rare_label_categories',
                 RareLabelEncoder(n_categories=1,
                                  variables=['Sex', 'Embarked', 'Title',
                                             'Cabin'])),
                ('categorical_encoder',
                 OneHotEncoder(drop_last=True,
                               variables=['Sex', 'Embarked', 'Title',
                                          'Cabin'])),
                ('scaler', StandardScaler()),
                ('logit', LogisticRegression(C=0.0005, random_state=0))])

In [14]:
class_ = titanic_pipeline.predict(X_test)
pred = titanic_pipeline.predict_proba(X_test)[:,1]

In [15]:
print('test roc-auc: {}'.format(roc_auc_score(y_test, pred)))
print('test accuracy: {}'.format(accuracy_score(y_test, class_)))
print()

test roc-auc: 0.8650635930047695
test accuracy: 0.6703910614525139



In [16]:
class_train = titanic_pipeline.predict(X_train)
pred_train = titanic_pipeline.predict_proba(X_train)[:,1]

In [17]:
print('train roc-auc: {}'.format(roc_auc_score(y_train, pred_train)))
print('train accuracy: {}'.format(accuracy_score(y_train, class_train)))
print()

train roc-auc: 0.8538021197880212
train accuracy: 0.6825842696629213

